In [2]:
import numpy as np
import pandas as pd

# numpy

## element wise comparison

In [125]:
(np.array([1, 2, 3]) == np.array([1, 2, 3])).all()

True

# pandas

## how to set values

In [12]:
df = pd.DataFrame({'item_id': [1, 1, 1, 2, 2, 2, 3, 3, 3],
                   'cluster_id': [0, 1, 2, 0, 1, 2, 0, 1, 2],
                   'cluster_size': [2, 3, 4, 4, 3, 2, 3, 3, 3]}).set_index(['item_id', 'cluster_id'])
df['label'] = 'unlabeled'

In [13]:
df

cluster_size      label
item_id cluster_id                         
1       0                      2  unlabeled
        1                      3  unlabeled
        2                      4  unlabeled
2       0                      4  unlabeled
        1                      3  unlabeled
        2                      2  unlabeled
3       0                      3  unlabeled
        1                      3  unlabeled
        2                      3  unlabeled

### via index

In [15]:
df.loc[(1, 2), 'label'] = 'foo'
df.loc[(1, 2), 'label']

'foo'

### via set_value

In [16]:
df.set_value((1, 2), 'label', 'bar')
df.loc[(1, 2), 'label']

'bar'

## concat text on group by
see http://stackoverflow.com/questions/37730843/how-to-concatenate-text-in-pandas-groupby

In [1]:
import string
import random

def text_generator(size=6, chars=string.ascii_lowercase):
    return ''.join(random.choice(chars) for _ in range(size))

In [2]:
%%time
items, clusters, texts = [], [], []
for item in range(200):
    for cluster in range(1000):
        for line in range(random.randint(1, 4)):
            items.append(item)
            clusters.append(cluster)
            texts.append(text_generator())
df = pd.DataFrame({'item_id': items, 'cluster_id': clusters, 'text': texts})
#df.loc[(0, 0), 'text'] = np.nan  # add nan to make things a little bit more interesting

CPU times: user 3.92 s, sys: 40.5 ms, total: 3.96 s
Wall time: 3.97 s


In [3]:
df['text'] = df.text.fillna('')

### use groupby + aggregation

In [4]:
grouped = df.groupby(('item_id', 'cluster_id'))

In [5]:
df_cluster = pd.DataFrame(grouped.size()).rename(columns={0: 'cluster_size'})

In [134]:
%%time
df_cluster['texts'] = grouped.text.agg(lambda x: ' '.join(x.dropna()))

CPU times: user 1min 22s, sys: 341 ms, total: 1min 22s
Wall time: 1min 22s


In [6]:
%%time
df_cluster['texts'] = grouped.text.agg(lambda x: ' '.join(x))

CPU times: user 9.67 s, sys: 51 ms, total: 9.73 s
Wall time: 9.76 s


In [15]:
#%%time
#df_cluster['texts'] = grouped.text.str.cat(sep=' ')

In [135]:
df_cluster.head()

cluster_size                        texts
item_id cluster_id                                           
0       0                      4         QYJ7VW 433ZCH 47K8FP
        1                      3         P3XM67 XTG2GT BS8S0P
        2                      4  TRL7B8 1CZMYX HFPI1B KS7J8Z
        3                      2                ILNM0D T378HL
        4                      3         O9AL54 ALG8E3 NVRNF2

### use python lists

In [8]:
def text_groupby(df, df_cluster, attr):
    text_lookup = {}
    df[attr] = df[attr].fillna('')
    columns = zip(df.item_id.values, df.cluster_id.values, df[attr].values)
    for item_id, cluster_id, text in columns:
        text_lookup.setdefault((item_id, cluster_id), []).append(text)
        
    item_ids, cluster_ids, all_texts = [], [], []
    for (item_id, cluster_id), texts in text_lookup.items():
        item_ids.append(item_id)
        cluster_ids.append(cluster_id)
        all_texts.append(' '.join(texts))
        
    df_tags = pd.DataFrame({'item_id': item_ids, 'cluster_id': cluster_ids,
                            'texts': all_texts}).set_index(['item_id', 'cluster_id'])
    df_cluster = df_cluster.merge(df_tags, left_index=True, right_index=True)
    return df_cluster

In [7]:
df_cluster = pd.DataFrame(grouped.size()).rename(columns={0: 'cluster_size'})

In [11]:
%%time
text_groupby(df, df_cluster, 'text').head()

CPU times: user 2.48 s, sys: 77.6 ms, total: 2.56 s
Wall time: 2.57 s


cluster_size                        texts
item_id cluster_id                                           
0       0                      3         ezpyln vdlxai qdclyt
        1                      4  rcdkkb gpgrnh crdiqk tedrjw
        2                      2                vcgsmq syaiwt
        3                      2                obumzn tnvfjo
        4                      3         nonndx pxgxii bggiaa

## apply function to multiple columns

In [18]:
lookup = {0: {'vcgsmq': 'blub'}}

In [21]:
%%time
def get_new_text(lookups, row):
    return lookups.get(row.item_id, {}).get(row.text, 'new_text')

df['new_text'] = df[['item_id', 'text']].apply(lambda row: get_new_text(lookup, row), axis=1)

CPU times: user 14.2 s, sys: 51 ms, total: 14.3 s
Wall time: 14.3 s


In [24]:
df[(df.item_id == 0) & (df.text == 'vcgsmq')]

,cluster_id,item_id,text,new_text
7,2,0,vcgsmq,blub
